In [1]:
from pyspark.sql import SparkSession, Row
from pyspark import SparkConf, SparkContext
import os
import pandas as pd
#os.environ['SPARK_HOME']="C:/Users/321ni/spark-2.4.3-bin-hadoop2.7/"
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
#os.environ['PYSPARK_DRIVER_PYTHON_OPS']='lab'
os.environ['PYSPARK_PYTHON']='python'
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
import findspark
#findspark.init("C:/Users/321ni/spark-2.4.3-bin-hadoop2.7/")
findspark.find()

'C:\\Users\\321ni\\spark-3.5.0-bin-hadoop3'

In [3]:
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "4g").appName("sparkan").getOrCreate()

In [4]:
mvcdata2 = spark.read.format('csv').options(header='true',inferschema='true').load("vehicle.csv")
#mvcdata2.show()

In [5]:
mvcdata2.show()

+---------+------------+----------+-------------------+--------------------+------------------+--------------------+--------------+-------------+------------+----------------+-----------------+----------+---------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+------------------+----------------+----------------------+---------------------------+---------------------+---------------------+
|UNIQUE_ID|COLLISION_ID|CRASH_DATE|         CRASH_TIME|          VEHICLE_ID|STATE_REGISTRATION|        VEHICLE_TYPE|  VEHICLE_MAKE|VEHICLE_MODEL|VEHICLE_YEAR|TRAVEL_DIRECTION|VEHICLE_OCCUPANTS|DRIVER_SEX|DRIVER_LICENSE_STATUS|DRIVER_LICENSE_JURISDICTION|           PRE_CRASH|     POINT_OF_IMPACT|      VEHICLE_DAMAGE|    VEHICLE_DAMAGE_1|  VEHICLE_DAMAGE_2|VEHICLE_DAMAGE_3|PUBLIC_PROPERTY_DAMAGE|PUBLIC_PROPERTY_DAMAGE_TYPE|CONTRIBUTING_FACTOR_1|CONTRIBUTING_FACTOR_2|
+---------+------------+----------+-------------------+-----

In [6]:
mvcdata2.createOrReplaceTempView("vData")

In [7]:
mvcdata2.count()

3704406

## Grouping Vehicles as Large, Medium and Small

In [8]:
def vehicle_classifier(row):
    
    formatted_row = ','.join([key if type(key)==str else str(key) for key in row])
    
    if "bus" in formatted_row.lower() or "truck" in formatted_row.lower() or "freig" in formatted_row.lower() \
    or "ceme" in formatted_row.lower() or "fire" in formatted_row.lower() or "000" == formatted_row.lower()\
    or "concrete mixer" in formatted_row.lower() or "com" in formatted_row.lower() or "crane" in formatted_row.lower()\
    or "frieg" in formatted_row.lower() or "fdny" in formatted_row.lower() or "fedex" in formatted_row.lower()\
    or "dump" in formatted_row.lower() or "deli" in formatted_row.lower() or "exca" in formatted_row.lower()\
    or "flat" in formatted_row.lower() or "gar" in formatted_row.lower() or "lift boom" in formatted_row.lower()\
    or "mini schoo" in formatted_row.lower() or "omnib" in formatted_row.lower() or "power" in formatted_row.lower()\
    or "ic corporation" in formatted_row.lower() or "schoo" in formatted_row.lower() or "r/v" in formatted_row.lower()or\
    "rv" in formatted_row.lower() or "multi-wh" in formatted_row.lower() or "mta" in formatted_row.lower() or "mack" in\
    formatted_row.lower() or "sanit" in formatted_row.lower() or ("snow" in formatted_row.lower() and "snowm" \
    not in formatted_row.lower()) or "tank" in formatted_row.lower() or "lunch wagon" in formatted_row.lower()\
    or "trail" in formatted_row.lower() or "const" in formatted_row.lower() or "cat" in formatted_row.lower() or\
    "trlr" in formatted_row.lower() or "winne" in formatted_row.lower():
        return row + Row(VEHICLE_GROUP = "Heavy")
    elif "van" in formatted_row.lower() or "fleet" in formatted_row.lower() or "usps" in formatted_row.lower() \
    or "am" in formatted_row.lower() or ('4' in formatted_row.lower() and 'wh' in formatted_row.lower())\
    or "chassis" in formatted_row.lower() or "carry all" in formatted_row.lower() or "limo" in formatted_row.lower() or\
    "miniv" in formatted_row.lower() or "open body" in formatted_row.lower() or "pick" in formatted_row.lower() or\
    "pk" in formatted_row.lower() or "post" in formatted_row.lower() or "courier" in formatted_row.lower() or \
    "mail" in formatted_row.lower() or ("nypd" in formatted_row.lower() and "tow" in formatted_row.lower()) \
    or "sub" in formatted_row.lower() or "stak" in formatted_row.lower() or "stree" in formatted_row.lower() or "sw"\
    in formatted_row.lower() or "tow" in formatted_row.lower() or "trac" in formatted_row.lower() or "sprin" in \
    formatted_row.lower() or "luv" in formatted_row.lower() or "gmc" in formatted_row.lower() or "lives"\
    in formatted_row.lower() or "almbulance" in formatted_row.lower() or "uti" in formatted_row.lower() or "yello"\
    in formatted_row.lower() or " yw po" == formatted_row.lower() or\
    ("u" in formatted_row.lower() and "ha" in formatted_row.lower()) or "ups" in formatted_row.lower() or \
    "utl" in formatted_row.lower():
        return row + Row(VEHICLE_GROUP = "Medium")
    elif "sed" in formatted_row.lower() or "car" in formatted_row.lower() or "0" == formatted_row.lower() or \
    ('3' in formatted_row.lower() and 'wh' in formatted_row.lower()) or "convertible" in formatted_row.lower()\
    or "delv" in formatted_row.lower() or "mini" in formatted_row.lower() or "pas" in formatted_row.lower()\
    or "pc" in formatted_row.lower() or ("3" in formatted_row.lower() and "doo" in formatted_row.lower())\
    or ("2" in formatted_row.lower() and "doo" in formatted_row.lower()) or ("nypd" in formatted_row.lower()\
    and "tow" not in formatted_row.lower()) or "station wa" in formatted_row.lower() or "suv" in formatted_row.lower()\
    or "taxi" in formatted_row.lower() or "wagon" in formatted_row.lower():
        return row + Row(VEHICLE_GROUP = "Light")
    elif "bicycle" in formatted_row.lower() or "bik" in formatted_row.lower() or "mope" in formatted_row.lower()\
    or "dirt" in formatted_row.lower() or "sco" in formatted_row.lower() or "fork" in formatted_row.lower() or \
    "dolly" in formatted_row.lower() or "motor" in formatted_row.lower() or \
    ("mo" in formatted_row.lower() and "p" in formatted_row.lower()) or ("2" in formatted_row.lower() and "wh" in\
    formatted_row.lower()) or "pallet" in formatted_row.lower() or "pedicab" in formatted_row.lower():
        return row + Row(VEHICLE_GROUP = "Small")
    elif "n/a" in formatted_row.lower() or "na" in formatted_row.lower() or "other" in formatted_row.lower() or\
    "unk" in formatted_row.lower() or "ukn" in formatted_row.lower():
        return row + Row(VEHICLE_GROUP = "Other")
    else:
        return row + Row(VEHICLE_GROUP = "Unspecified")

In [9]:
vehicle_types_sql = "SELECT COLLISION_ID,VEHICLE_TYPE,VEHICLE_MAKE FROM vData WHERE VEHICLE_TYPE NOT LIKE 'NULL' OR VEHICLE_MAKE NOT LIKE 'NULL' ORDER BY COLLISION_ID"

In [10]:
vehicle_types_df = spark.sql(vehicle_types_sql)

In [11]:
vehicle_types_df.count()

3541338

In [12]:
vehicle_type_rdd = vehicle_types_df.rdd.map(vehicle_classifier).filter(lambda x: x is not None)

schema = StructType([
    StructField("COLLISION_ID", IntegerType(), True),
    StructField("VEHICLE_TYPE", StringType(), True),
    StructField("VEHICLE_MAKE", StringType(), True),
    StructField("VEHICLE_GROUP", StringType(), True)
])
vehicle_type_df_final = spark.createDataFrame(vehicle_type_rdd,schema=schema)

In [13]:
vehicle_type_df_final.show(150)

+------------+--------------------+------------+-------------+
|COLLISION_ID|        VEHICLE_TYPE|VEHICLE_MAKE|VEHICLE_GROUP|
+------------+--------------------+------------+-------------+
|          22|                 VAN|        NULL|       Medium|
|          22|SPORT UTILITY / S...|        NULL|       Medium|
|          23|                TAXI|        NULL|        Light|
|          23|                TAXI|        NULL|        Light|
|          24|                 BUS|        NULL|        Heavy|
|          24|   PASSENGER VEHICLE|        NULL|        Light|
|          25|   PASSENGER VEHICLE|        NULL|        Light|
|          25|   PASSENGER VEHICLE|        NULL|        Light|
|          26|   PASSENGER VEHICLE|        NULL|        Light|
|          26|   PASSENGER VEHICLE|        NULL|        Light|
|          27|                TAXI|        NULL|        Light|
|          27|   PASSENGER VEHICLE|        NULL|        Light|
|          28|                TAXI|        NULL|       

In [14]:
vehicle_type_df_final = vehicle_type_df_final.where(vehicle_type_df_final["VEHICLE_GROUP"] != "Unspecified")

In [15]:
vehicle_type_df_final.createOrReplaceTempView("vcData")

In [16]:
vehicle_collision_type_sql = """
SELECT
  SUM(CASE UPPER(VEHICLE_GROUP) WHEN 'SMALL' THEN 1 ELSE 0 END) as small_count,
  SUM(CASE UPPER(VEHICLE_GROUP) WHEN 'LIGHT' THEN 1 ELSE 0 END) as light_count,
  SUM(CASE UPPER(VEHICLE_GROUP) WHEN 'MEDIUM' THEN 1 ELSE 0 END) as medium_count,
  SUM(CASE UPPER(VEHICLE_GROUP) WHEN 'HEAVY' THEN 1 ELSE 0 END) as heavy_count,
  SUM(CASE UPPER(VEHICLE_GROUP) WHEN 'OTHER' THEN 1 ELSE 0 END) as other_count,
  COLLISION_ID
FROM vcData
GROUP BY COLLISION_ID;
"""

In [17]:
vehicle_collision_type_df = spark.sql(vehicle_collision_type_sql)

In [18]:
vehicle_collision_type_df.count()

1836239

In [19]:
vehicle_collision_type_df.show()

+-----------+-----------+------------+-----------+-----------+------------+
|small_count|light_count|medium_count|heavy_count|other_count|COLLISION_ID|
+-----------+-----------+------------+-----------+-----------+------------+
|          0|          0|           0|          1|          0|         148|
|          1|          1|           0|          0|          0|         463|
|          0|          1|           1|          0|          0|         471|
|          0|          1|           0|          1|          0|         496|
|          0|          2|           0|          0|          0|         833|
|          0|          1|           1|          0|          0|        1088|
|          0|          1|           0|          0|          0|        1238|
|          0|          0|           1|          0|          1|        1342|
|          0|          1|           1|          0|          0|        1580|
|          0|          0|           1|          0|          1|        1591|
|          0

In [20]:
def vehicle_collision_detector(row):
    s = ""
    st = ""
    if row[0]>0:
        s+="S"*row[0]
        st+="S"
    if row[1]>0:
        s+="L"*row[1]
        st+="L"
    if row[2]>0:
        s+="M"*row[2]
        st+="M"
    if row[3]>0:
        s+="H"*row[3]
        st+="H"
    if row[4]>0:
        s+="O"*row[4]
        st+="O"
    return row+Row(ACC_IN_BETWEEN = s)+Row(ACC_IN_BETWEEN_UNQ = st)

In [21]:
vehicle_collision_type_rdd = vehicle_collision_type_df.rdd.map(vehicle_collision_detector)
vehicle_collision_type_rdd.take(5)

[(0, 0, 0, 1, 0, 148, 'H', 'H'),
 (1, 1, 0, 0, 0, 463, 'SL', 'SL'),
 (0, 1, 1, 0, 0, 471, 'LM', 'LM'),
 (0, 1, 0, 1, 0, 496, 'LH', 'LH'),
 (0, 2, 0, 0, 0, 833, 'LL', 'L')]

In [22]:
schema = StructType([
    StructField("small_count", IntegerType(), True),
    StructField("light_count", IntegerType(), True),
    StructField("medium_count", IntegerType(), True),
    StructField("heavy_count", IntegerType(), True),
    StructField("other_count", IntegerType(), True),
    StructField("COLLISION_ID", IntegerType(), True),
    StructField("ACC_IN_BETWEEN", StringType(), True),
    StructField("ACC_IN_BETWEEN_UNQ", StringType(), True)
])
vehicle_collision_type_df_final = spark.createDataFrame(vehicle_collision_type_rdd,schema=schema)

In [23]:
vehicle_collision_type_df_final.show()

+-----------+-----------+------------+-----------+-----------+------------+--------------+------------------+
|small_count|light_count|medium_count|heavy_count|other_count|COLLISION_ID|ACC_IN_BETWEEN|ACC_IN_BETWEEN_UNQ|
+-----------+-----------+------------+-----------+-----------+------------+--------------+------------------+
|          0|          0|           0|          1|          0|         148|             H|                 H|
|          1|          1|           0|          0|          0|         463|            SL|                SL|
|          0|          1|           1|          0|          0|         471|            LM|                LM|
|          0|          1|           0|          1|          0|         496|            LH|                LH|
|          0|          2|           0|          0|          0|         833|            LL|                 L|
|          0|          1|           1|          0|          0|        1088|            LM|                LM|
|         

In [24]:
vehicle_collision_type_df_final = vehicle_collision_type_df_final.groupBy("ACC_IN_BETWEEN_UNQ").count().withColumnRenamed("count","GROUPS")

In [25]:
vehicle_collision_type_df_final.count()

28

In [26]:
vehicle_collision_type_df_final_part1 = vehicle_collision_type_df_final.orderBy("GROUPS").limit(19)
vehicle_collision_type_df_final_part2 = vehicle_collision_type_df_final.orderBy("GROUPS").\
subtract(vehicle_collision_type_df_final_part1).orderBy("GROUPS").limit(10)

In [27]:
vehicle_collision_type_df_final_part1.show()

+------------------+------+
|ACC_IN_BETWEEN_UNQ|GROUPS|
+------------------+------+
|               SHO|    10|
|              SLMO|    17|
|              SLMH|    24|
|               SMO|    38|
|              LMHO|    58|
|               SLO|    91|
|               SMH|   116|
|               MHO|   173|
|               SLH|   189|
|               LHO|   328|
|               SLM|   928|
|                SO|  2840|
|               LMO|  2907|
|                SH|  3880|
|                HO|  5461|
|               LMH|  7451|
|                 S| 11860|
|                 O| 18625|
|                SM| 22369|
+------------------+------+



In [28]:
vehicle_collision_type_df_final_part2.show()

+------------------+------+
|ACC_IN_BETWEEN_UNQ|GROUPS|
+------------------+------+
|                MO| 33492|
|                SL| 39199|
|                 H| 40299|
|                LO| 78748|
|                MH| 85437|
|                LH|152135|
|                 M|280592|
|                LM|449335|
|                 L|599637|
+------------------+------+



In [ ]:
vehicle_collision_type_df_final_pandas = vehicle_collision_type_df_final.toPandas()

fig = px.pie(vehicle_collision_type_df_final_pandas, names="ACC_IN_BETWEEN_UNQ", values="GROUPS", title="Collisions")
fig.update_layout(height=800, width=800)
fig.show()

In [ ]:
vehicle_collision_type_df_final_part1_pandas = vehicle_collision_type_df_final_part1.toPandas()
vehicle_collision_type_df_final_part2_pandas = vehicle_collision_type_df_final_part2.toPandas()



fig = px.pie(vehicle_collision_type_df_final_part1_pandas, names="ACC_IN_BETWEEN_UNQ", values="GROUPS", title="Collisions")
fig.update_layout(height=800, width=800)
fig.show()

fig2 = px.pie(vehicle_collision_type_df_final_part2_pandas, names="ACC_IN_BETWEEN_UNQ", values="GROUPS", title="Collisions")
fig2.update_layout(height=800, width=800)
fig2.show()

In [ ]:
vehicle_groups_df = vehicle_type_df_final.groupBy("VEHICLE_GROUP").count().withColumnRenamed("count","ACCIDENTS").orderBy("VEHICLE_GROUP")
vehicle_groups_df.show()

In [ ]:
import plotly.express as px

vehicle_group_pandas = vehicle_groups_df.toPandas()

# Plot the data using Plotly Express
fig = px.histogram(vehicle_group_pandas, x="VEHICLE_GROUP", y="ACCIDENTS", title="Vehicle Group vs Accidents")
fig.show()

# Vehicle Manafucturing Year classification

In [ ]:
yearly_vehicle_sql = "SELECT VEHICLE_YEAR, COUNT(*) AS YEARLY_COUNT FROM vData WHERE VEHICLE_YEAR NOT LIKE 'NULL' \
AND VEHICLE_YEAR > '1111' AND VEHICLE_YEAR < '2025'\
GROUP BY VEHICLE_YEAR ORDER BY VEHICLE_YEAR"

In [ ]:
yearly_vehicle_df = spark.sql(yearly_vehicle_sql)

In [ ]:
yearly_vehicle_df.show()

In [ ]:
vehicle_year_pandas = yearly_vehicle_df.toPandas()

# Plot the data using Plotly Express
fig = px.line(vehicle_year_pandas, x="VEHICLE_YEAR", y="YEARLY_COUNT", title="Vehicle Year vs Accidents")
fig.show()

# Contributing Factor Analysis of Unlicensed Drivers

In [ ]:
dl_status_sql = """
    SELECT CONTRIBUTING_FACTOR_1, COUNT(*) AS ACCIDENTS
    FROM (
    SELECT CONTRIBUTING_FACTOR_1, DRIVER_LICENSE_STATUS FROM vData WHERE DRIVER_LICENSE_STATUS LIKE 'Unlicensed'
    )
    WHERE CONTRIBUTING_FACTOR_1 NOT LIKE 'Unspecified'
    GROUP BY CONTRIBUTING_FACTOR_1
    ORDER BY CONTRIBUTING_FACTOR_1;
"""

In [ ]:
dl_status_df = spark.sql(dl_status_sql).na.drop()
dl_status_df.show(150)

In [ ]:
dl_status_pandas = dl_status_df.toPandas()

# Plot the data using Plotly Express
fig = px.pie(dl_status_pandas, names="CONTRIBUTING_FACTOR_1", values="ACCIDENTS", title="Contributing Factors")
fig.update_layout(height=1000, width=1000)
fig.show()